In [1]:
import os
import utils
import numpy as np
import time

# Download all data from Figshare
### Make sure there is at least 1320 GB empty space (420GB for raw data + 900GB for processed data)

In [ ]:
root = r'E:\Zhong-et-2025' # modify this to be your local folders, for example

utils.download_data_from_figshare(root)  

Downloading...
E:\Zhong-et-2025\beh\Beh_naive_test1.npy
Beh_naive_test1.npy
E:\Zhong-et-2025\beh\Beh_naive_test2.npy
Beh_naive_test2.npy
E:\Zhong-et-2025\beh\Beh_naive_test3.npy
Beh_naive_test3.npy
E:\Zhong-et-2025\beh\Beh_sup_test1.npy
Beh_sup_test1.npy
E:\Zhong-et-2025\beh\Beh_sup_test2.npy
Beh_sup_test2.npy
E:\Zhong-et-2025\beh\Beh_sup_test3.npy
Beh_sup_test3.npy
E:\Zhong-et-2025\beh\Beh_sup_train1_after_learning.npy
Beh_sup_train1_after_learning.npy
E:\Zhong-et-2025\beh\Beh_sup_train1_before_learning.npy
Beh_sup_train1_before_learning.npy
E:\Zhong-et-2025\beh\Beh_sup_train2_after_learning.npy
Beh_sup_train2_after_learning.npy
E:\Zhong-et-2025\beh\Beh_sup_train2_before_learning.npy
Beh_sup_train2_before_learning.npy
E:\Zhong-et-2025\beh\Beh_test1_after_grating.npy
Beh_test1_after_grating.npy
E:\Zhong-et-2025\beh\Beh_test1_before_grating.npy
Beh_test1_before_grating.npy
E:\Zhong-et-2025\beh\Beh_test2_after_grating.npy
Beh_test2_after_grating.npy
E:\Zhong-et-2025\beh\Beh_test2_before_

TX61_2021_06_23_trans.npz
E:\Zhong-et-2025\retinotopy\TX61_2021_06_25_trans.npz
TX61_2021_06_25_trans.npz
E:\Zhong-et-2025\retinotopy\TX83_2022_08_29_trans.npz
TX83_2022_08_29_trans.npz
E:\Zhong-et-2025\retinotopy\TX85_2022_06_14_trans.npz
TX85_2022_06_14_trans.npz
E:\Zhong-et-2025\retinotopy\TX85_2022_06_17_trans.npz
TX85_2022_06_17_trans.npz
E:\Zhong-et-2025\retinotopy\TX88_2022_06_13_trans.npz
TX88_2022_06_13_trans.npz
E:\Zhong-et-2025\retinotopy\VR2_2021_05_04_trans.npz
VR2_2021_05_04_trans.npz
E:\Zhong-et-2025\retinotopy\TX88_2022_06_20_trans.npz
TX88_2022_06_20_trans.npz
E:\Zhong-et-2025\retinotopy\TX88_2022_07_15_trans.npz
TX88_2022_07_15_trans.npz
E:\Zhong-et-2025\retinotopy\TX88_2022_07_19_trans.npz
TX88_2022_07_19_trans.npz
E:\Zhong-et-2025\retinotopy\TX88_2022_07_22_trans.npz
TX88_2022_07_22_trans.npz
E:\Zhong-et-2025\retinotopy\VR2_2021_03_20_trans.npz
VR2_2021_03_20_trans.npz
E:\Zhong-et-2025\retinotopy\VR2_2021_04_06_trans.npz
VR2_2021_04_06_trans.npz
E:\Zhong-et-2025\ret

## Data process
### takes about 8 hours

In [ ]:
# load experiment information for imaging mice
exp_info = np.load(os.path.join(root, r'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()

In [2]:
start_time = time.time()

In [3]:
# create a subfolder named "process_data" under root if it doesn't exist.
proc_root = os.path.join(root, 'process_data')
if not os.path.isdir(proc_root):
    os.makedirs(proc_root)

In [ ]:
# get interpolational neural activity (neurons * trials * positions)
# bins of positon: 60, each bin is 1 decimeter, total length of corridor is 6 meters 

for exp_type in exp_info.keys():
    print(exp_type)
    Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
    db = exp_info[exp_type] # get experiment information
    n = 1
    for ndb in db:
        if 'stimtype' in ndb.keys():
            kn = '%s_%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'], ndb['stimtype'])
        else:
            kn = '%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'])
        save_name = os.path.join(root, 'process_data', '%s_%s_%s_interpolate_spk.npy'%(ndb['mname'], ndb['datexp'], ndb['blk']))
        if os.path.exists(save_name):
            print("File has been created")
        else:    
            beh = Beh[kn]
            spk = utils.load_spk(ndb, root=os.path.join(root, 'spk'))
            nneu, nfr = spk.shape  

            ntrials, CL = beh['ntrials'], beh['Corridor_Length']    
            VRmove = beh['ft_move'][:nfr]>0
            ft_AcumPos = beh['ft_PosCum'][:nfr]

            utils.get_interpPos_spk(spk[:, VRmove], ft_AcumPos[VRmove], ntrials, n_bins=60, 
                                    lengths=CL, save_path=save_name)
        print('done %d'%(n))
        n += 1

unsup_train1_before_learning


## Get stimulus-selective index (dprime)

In [ ]:
stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']
exps = ['unsup_train1_before_learning', 'unsup_train1_after_learning', 
        'sup_train1_before_learning',   'sup_train1_after_learning', 
        'train1_before_grating',        'train1_after_grating', 
        
        'naive_test1', 'unsup_train2_before_learning', 'sup_train2_before_learning', 
        'unsup_train2_after_learning', 'sup_train2_after_learning',
        
        'naive_test1', 'unsup_train2_after_learning', 'sup_train2_after_learning',  'test1_after_grating']

Stim_ID = [[2, 0], [2, 0], # leaf1 vs circle1
           [2, 0], [2, 0],
           [2, 0], [2, 0],
          
           [3, 0], [3, 0], [3, 0],  # leaf2 vs circle1
           [3, 0], [3, 0],
           
           [2, 3], [2, 3], [2, 3], [2, 3]] #  leaf1 vs leaf2

for exp_type, SID in zip(exps, Stim_ID):
    print(exp_type)
    print(SID)
    db = exp_info[exp_type] # get experiment information
    Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item() # load behavior    
    all_dat = utils.Get_dprime_selective_neuron(db, Beh, stim_ID=SID, root=root) # get dprime
    # save the dprime
    fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[SID[0]], stim[SID[1]])
    np.save(os.path.join(root, 'process_data', fn), all_dat)
    print(fn)    

## Get density map of stimulus-selective neurons 

In [ ]:
exps = ['unsup_train1_before_learning', 'unsup_train1_after_learning', 
        'sup_train1_before_learning',   'sup_train1_after_learning', 
        
        'naive_test1', 'unsup_train2_before_learning', 'sup_train2_before_learning', 
        'unsup_train2_after_learning', 'sup_train2_after_learning',
        
        'naive_test1', 'unsup_train2_after_learning', 'sup_train2_after_learning']

Stim_ID = [[2, 0], [2, 0], # leaf1 vs circle1
           [2, 0], [2, 0],
          
           [3, 0], [3, 0], [3, 0],  # leaf2 vs circle1
           [3, 0], [3, 0],
           
           [2, 3], [2, 3], [2, 3]] #  leaf1 vs leaf2

types = ['both', 'both', # include neurons selective to either leaf1 or circle1 in the case of leaf1 vs circle1
         'both', 'both',
        
         'stim1', 'stim1', 'stim1', # include neurons selective to leaf1 in the case of leaf1 vs circle1
         'stim1', 'stim1',
        
         'both', 'both', 'both']

stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']

for exp_type, SID, typ in zip(exps, Stim_ID, types):
    print(exp_type)
    fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[SID[0]], stim[SID[1]])
    # load saved dprime
    dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item()
    dprime, retinotopy = dprimes['dprime'], dprimes['retinotopy']

    # get density map
    sel_map = utils.Get_density_map(dprime, retinotopy, dp_thr=0.3, typ=typ)
    if typ=='stim1':
        saveN = fn.split('.')[0] + '_%s_distribution.npy'%(stim[SID[0]])
    elif typ=='stim2':
        saveN = fn.split('.')[0] + '_%s_distribution.npy'%(stim[SID[1]])
    else:
        saveN = fn.split('.')[0] + '_distribution.npy'
    np.save(os.path.join(root, 'process_data', saveN), sel_map)
    print(saveN)    

## Percentage of stimulus-selective neurons

In [ ]:
exps = ['unsup_train1_before_learning', 'unsup_train1_after_learning', 
        'sup_train1_before_learning',   'sup_train1_after_learning', 
        'train1_before_grating',        'train1_after_grating', 
        
        'unsup_train2_before_learning', 'sup_train2_before_learning', 
        'unsup_train2_after_learning', 'sup_train2_after_learning',
        
        'naive_test1', 'unsup_train2_after_learning', 'sup_train2_after_learning',  'test1_after_grating']

Stim_ID = [[2, 0], [2, 0], # leaf1 vs circle1
           [2, 0], [2, 0],
           [2, 0], [2, 0],
          
           [3, 0], [3, 0],  # leaf2 vs circle1
           [3, 0], [3, 0],
           
           [2, 3], [2, 3], [2, 3], [2, 3]] #  leaf1 vs leaf2

stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']

for exp_type, SID in zip(exps, Stim_ID):
    print(exp_type)
    fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[SID[0]], stim[SID[1]])
    
    dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item() # load saved dprime
    dprime, retinotopy = dprimes['dprime'], dprimes['retinotopy']

    frac = utils.Get_selective_neuron_fraction_with_dprime(dprime, retinotopy)
    # save the fraction of selective neurons 
    saveN = fn.split('.')[0] + '_frac.npy'
    np.save(os.path.join(root, 'process_data', saveN), frac)
    print(saveN)

## Percentage and distribution of reward prediciton neurons

In [ ]:
exps = ['unsup_train1_before_learning', 'unsup_train1_after_learning', 
        'sup_train1_before_learning',   'sup_train1_after_learning']

for exp_type in exps:
    print(exp_type)
    db = exp_info[exp_type] # get experiment information
    fn = '%s_leaf1_circle1_dprime.npy'%(exp_type)
    Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item() # load behavior
    dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item() # load saved stimulus selective dprime
    frac, imgs = utils.Get_dprime_rewPred_neuron(db, Beh, dprimes, root=root, 
                                  load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'), dp_thr=0.3)
    # save the fraction of reward prediciton neurons 
    save_fn0 = '%s_rew_frac.npy'%(exp_type)
    save_fn1 = '%s_rew_distribution.npy'%(exp_type)
    np.save(os.path.join(root, 'process_data', save_fn0), frac)
    np.save(os.path.join(root, 'process_data', save_fn1), imgs)    

## Coding direction

In [ ]:
stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']
exps = ['naive_test1', 'unsup_test1', 'sup_test1', 
        
        'unsup_train2_after_learning', 'sup_train2_after_learning', 'test1_after_grating',
        
        'naive_test2', 'unsup_test2', 'sup_test2', 'test2_after_grating']

Stim_ID = [[2, 0], [2, 0], [2, 0], # leaf1 vs circle1
          
           [2, 0], [2, 0], [2, 0],  
           
           [2, 3], [2, 3], [2, 3], [2, 3]] #  leaf1 vs leaf2

for exp_type, SID in zip(exps, Stim_ID):
    print(exp_type)
    db = exp_info[exp_type] # get experiment information
    Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
    dat = utils.Get_coding_direction(db, Beh, stim_ref=SID, prc=5, root=root, 
                                load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'), n_bef=10)

    save_fn = '%s_coding_direction.npy'%(exp_type)
    np.save(os.path.join(root, 'process_data', save_fn), dat)    

## Positional tunning of stimulus-selective neurons, with train and test for cross validation

In [ ]:
exps = ['naive_test1', 'unsup_test1', 'sup_test1', 'naive_test3', 'unsup_test3', 'sup_test3']
Stim_ID = [2, 0]# leaf1 vs circle1

for exp_type in exps:
    print(exp_type)
    db = exp_info[exp_type] # get experiment information
    Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
    dat = utils.Get_sort_spk(db, Beh, stim_ref=Stim_ID, prc=5, root=root, 
                                load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'))
    save_fn = '%s_sort_spk.npy'%(exp_type)
    np.save(os.path.join(root, 'process_data', save_fn), dat)
    
# use 1/2 of trials (train) for getting dprime to define stimulus selective neurons;
# from the rest 1/2 trials, use half (tes1, 1/4 of total trials) to get and sort the peak positions
# sort the last 1/4 trials (test2) by the sorted index from test1

## Example stimulus selective neurons

In [ ]:
## 1. Use 1/2 of trials (train) for getting dprime to define stimulus selective neurons;
## 2. Return the activity of other 1/2 of trials (test) 

exp_type = 'sup_train1_after_learning' # type of experiment
db = exp_info[exp_type] # get experiment information
ndb = db[2]

Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
dat = utils.get_stimNeu_and_sorted(ndb, Beh['%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'])], 
                             stim_ref=[2, 0], thr=0.3, root=root, load_save_interp_spk=1)
save_fn = '%s_%s_%s_stimSelNeu_sorted.npy'%(ndb['mname'], ndb['datexp'], ndb['blk'])
np.save(os.path.join(root, 'process_data', save_fn), dat)

## Reward prediction neural responses; with 10 folds cross validation

In [ ]:
exps = ['sup_test1', 'sup_test2', 'sup_test3']
for exp_type in exps:
    print(exp_type)
    db = exp_info[exp_type] # get experiment information

    Beh = utils.load_exp_beh(root, exp_type)
    dat = utils.get_kfold_reward_response(root, db, Beh)

    save_fn = '%s_reward_response.npy'%(exp_type)
    np.save(os.path.join(root, 'process_data', save_fn), dat)

## Example reward prediction neurons

In [ ]:
exp_type = 'sup_test1' # type of experiment
db = exp_info[exp_type][-1] # get experiment information

Beh = utils.load_exp_beh(root, exp_type)
dat = utils.get_reward_neuorns(root, db, Beh['%s_%s_%s'%(db['mname'], db['datexp'], db['blk'])])

save_fn = 'Example_reward_neurons_in_%s.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn), dat)

In [ ]:
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")